In [1]:
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [2]:
import torch
from transformers import AutoTokenizer, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
def create_sbert_mpnet():
  model = "sentence-transformers/all-mpnet-base-v2"
  device = "cuda" if torch.cuda.is_available() else "cpu"
  return HuggingFaceEmbeddings(model_name = model, model_kwargs = {"device": device})

In [4]:
def create_flan_t5_base(load_in_8bit):
  model = "google/flan-t5-xl"
  tokenizer = AutoTokenizer.from_pretrained(model)
  return pipeline(
      task = "text2text-generation",
      model = model,
      tokenizer = tokenizer,
      max_new_tokens = 100,
      device_map = "auto",
      model_kwargs = {"load_in_8bit": load_in_8bit,
                      "temperature": 0.95}
  )

In [5]:
embedding = create_sbert_mpnet()
llm = create_flan_t5_base(load_in_8bit = False)

/tmp/ipython-input-3-1598665095.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name = model, model_kwargs = {"device": device})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
_question = "What is Grant of Patent in ArangoDB License?"
prompt = "Answer the following question. \n" + _question
res = llm(prompt)
print(res)

[{'generated_text': 'Grant of Patent in ArangoDB License Grant of Patent in ArangoDB License is a grant of patent to ArangoDB.'}]


In [9]:
pdf_path = "https://arangodb.com/wp-content/uploads/2023/09/Download-Contributor-License-Agreement.pdf"
loader = PDFPlumberLoader(pdf_path)
documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size = 200, chunk_overlap = 10)
texts = text_splitter.split_documents(documents)
persist_directory = "./chromadb"
vectordb = Chroma.from_documents(documents = texts, embedding = embedding, persist_directory = persist_directory)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [10]:
print(len(texts))

11


In [11]:
for t in texts[:4]:
  print(t)
  print("##**##" * 20)

page_content='ArangoDB GmbH
Individual Contributor License Agreement ("Agreement")
http://www.arangodb.com
Thank you for your interest in ArangoDB GmbH ("ArangoDB"). In order to
clarify the intellectual property license granted with Contributions
from any person or entity, ArangoDB must have a Contributor License
Agreement ("CLA") on file that has been signed by each Contributor,
indicating agreement to the license terms below. This license is for
your protection as a Contributor as well as the protection of ArangoDB
and its users; it does not change your rights to use your own
Contributions for any other purpose.
If you have not already done so, please complete and sign, then scan
and email a pdf file of this Agreement to cla@arangodb.com.
Alternatively, you may send it by facsimile to ArangoDB at +49-221' metadata={'source': 'https://arangodb.com/wp-content/uploads/2023/09/Download-Contributor-License-Agreement.pdf', 'file_path': 'https://arangodb.com/wp-content/uploads/2023/09/Downl

In [13]:
vdb = vectordb.get(include = ['embeddings', 'documents', 'metadatas'])
num = 2
print(vdb['documents'][:num])
print(vdb['embeddings'][:num])
print(vdb['metadatas'][:num])

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


['ArangoDB GmbH\nIndividual Contributor License Agreement ("Agreement")\nhttp://www.arangodb.com\nThank you for your interest in ArangoDB GmbH ("ArangoDB"). In order to\nclarify the intellectual property license granted with Contributions\nfrom any person or entity, ArangoDB must have a Contributor License\nAgreement ("CLA") on file that has been signed by each Contributor,\nindicating agreement to the license terms below. This license is for\nyour protection as a Contributor as well as the protection of ArangoDB\nand its users; it does not change your rights to use your own\nContributions for any other purpose.\nIf you have not already done so, please complete and sign, then scan\nand email a pdf file of this Agreement to cla@arangodb.com.\nAlternatively, you may send it by facsimile to ArangoDB at +49-221', 'ile to ArangoDB at +49-221-\n2722999-88. If necessary, send an original signed Agreement to\nArangoDB GmbH, Hohenstaufenring 43-45, 50674 Cologne, Germany\nPlease read this docum

In [14]:
hf_llm = HuggingFacePipeline(pipeline = llm)
retriever = vectordb.as_retriever(search_kwargs = {"k": 1})
qa = RetrievalQA.from_chain_type(llm = hf_llm, chain_type = "stuff", retriever = retriever)

question_t5_template = """
context: {context}
question: {question}
answer:
"""
QUESTION_T5_PROMPT = PromptTemplate(
    template = question_t5_template, input_variables = ["context", "question"]
)
qa.combine_documents_chain.llm_chain.prompt = QUESTION_T5_PROMPT

/tmp/ipython-input-14-2960683294.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(pipeline = llm)


In [15]:
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query": _question,})

/tmp/ipython-input-15-2547477848.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa({"query": _question,})
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'What is Grant of Patent in ArangoDB License?',
 'result': 'a perpetual, worldwide, non-exclusive, no-charge, royalty-free, irrevocable (except as stated in this section) patent license to make, have made, use, offer to sell, sell, import, and otherwise transfer the Work',
 'source_documents': [Document(metadata={'source': 'https://arangodb.com/wp-content/uploads/2023/09/Download-Contributor-License-Agreement.pdf', 'CreationDate': "D:20151116212836Z00'00'", 'page': 1, 'Subject': '', 'total_pages': 4, 'Title': 'Ohne Titel', 'Author': 'Claudius Weinberger', 'Creator': 'Word', 'file_path': 'https://arangodb.com/wp-content/uploads/2023/09/Download-Contributor-License-Agreement.pdf', 'Producer': 'Mac OS X 10.11.1 Quartz PDFContext', 'ModDate': "D:20151116212836Z00'00'", 'Keywords': ''}, page_content='grant to ArangoDB and to recipients of software distributed by\nArangoDB a perpetual, worldwide, non-exclusive, no-charge,\nroyalty-free, irrevocable (except as stated in this section

In [16]:
retriever.get_relevant_documents(_question)

/tmp/ipython-input-16-2308568279.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(_question)


[Document(metadata={'source': 'https://arangodb.com/wp-content/uploads/2023/09/Download-Contributor-License-Agreement.pdf', 'total_pages': 4, 'Creator': 'Word', 'page': 1, 'Title': 'Ohne Titel', 'ModDate': "D:20151116212836Z00'00'", 'Subject': '', 'file_path': 'https://arangodb.com/wp-content/uploads/2023/09/Download-Contributor-License-Agreement.pdf', 'Producer': 'Mac OS X 10.11.1 Quartz PDFContext', 'Keywords': '', 'CreationDate': "D:20151116212836Z00'00'", 'Author': 'Claudius Weinberger'}, page_content='grant to ArangoDB and to recipients of software distributed by\nArangoDB a perpetual, worldwide, non-exclusive, no-charge,\nroyalty-free, irrevocable (except as stated in this section)\npatent license to make, have made, use, offer to sell, sell,\nimport, and otherwise transfer the Work, where such license\napplies only to those patent claims licensable by You that are\nnecessarily infringed by Your Contribution(s) alone or by\ncombination of Your Contribution(s) with the Work to whi

In [17]:
ret = retriever.invoke(_question)
qa.combine_documents_chain._get_inputs(ret)

{'context': 'grant to ArangoDB and to recipients of software distributed by\nArangoDB a perpetual, worldwide, non-exclusive, no-charge,\nroyalty-free, irrevocable (except as stated in this section)\npatent license to make, have made, use, offer to sell, sell,\nimport, and otherwise transfer the Work, where such license\napplies only to those patent claims licensable by You that are\nnecessarily infringed by Your Contribution(s) alone or by\ncombination of Your Contribution(s) with the Work to which such\nContribution(s) was submitted. If any entity institutes patent\nlitigation against You or any other entity (including a cross-\nclaim or counterclaim in a lawsuit) alleging that your\nContribution, or the Work to which you have contributed,\nconstitutes direct or contributory patent infringement, then any\npatent licenses granted to that entity under this Agreement for'}

In [18]:
QUESTION_T5_PROMPT.format(question = _question, context = ret[0].page_content)

'\ncontext: grant to ArangoDB and to recipients of software distributed by\nArangoDB a perpetual, worldwide, non-exclusive, no-charge,\nroyalty-free, irrevocable (except as stated in this section)\npatent license to make, have made, use, offer to sell, sell,\nimport, and otherwise transfer the Work, where such license\napplies only to those patent claims licensable by You that are\nnecessarily infringed by Your Contribution(s) alone or by\ncombination of Your Contribution(s) with the Work to which such\nContribution(s) was submitted. If any entity institutes patent\nlitigation against You or any other entity (including a cross-\nclaim or counterclaim in a lawsuit) alleging that your\nContribution, or the Work to which you have contributed,\nconstitutes direct or contributory patent infringement, then any\npatent licenses granted to that entity under this Agreement for\nquestion: What is Grant of Patent in ArangoDB License?\nanswer:\n'